In [2]:
macro timeOften(ex::Expr)
    :(@time begin
     for i = 1:100000
      $ex
     end
    end)
end

## Type stability
### Don't make the compiler guess return types.  

In [109]:
function T(x::Real)
    if typeof(x) <: Integer
        return x^2 + 0.1
    else
        return Int(ceil(x))
    end
end

T (generic function with 2 methods)

In [126]:
T(x::Integer) = x^2 + 0.1

T (generic function with 3 methods)

In [125]:
methods(T)

# 2 methods for generic function "T":
T(x::Real) at In[109]:2
T(x) at In[18]:2

In [110]:
[ typeof(T(x)) for x in (1, 1.0) ]

2-element Array{Union{Type{Float64},Type{Int64}},1}:
 Float64
 Int64  

In [118]:
@timeOften T(1.0)
@timeOften T(1)

  0.000635 seconds
  0.000658 seconds (100.00 k allocations: 1.526 MB)


In [149]:
@code_llvm for i in 1:10000
         T(i)
        end
    

LoadError: LoadError: expression is not a function call, or is too complex for @code_llvm to analyze; break it down to simpler parts if possible
while loading In[149], in expression starting on line 1

In [158]:
@time x=T(1)

  0.000002 seconds (5 allocations: 176 bytes)


1.1

In [138]:
@code_llvm T(1)


define double @julia_T_22226(i64) {
top:
  %1 = mul i64 %0, %0
  %2 = sitofp i64 %1 to double
  %3 = fadd double %2, 1.000000e-01
  ret double %3
}


In [116]:
@code_warntype T(1)

Variables:
  x::Int64

Body:
  begin  # In[109], line 2:
      unless (Main.typeof)(x::Int64)::Type{Int64} <: Main.Integer::Bool goto 0 # In[109], line 3:
      return (Base.box)(Base.Float64,(Base.add_float)((Base.box)(Float64,(Base.sitofp)(Float64,(Base.box)(Int64,(Base.mul_int)(x::Int64,x::Int64)))),0.1))
      goto 1
      0:  # In[109], line 5:
      return x::Int64
      1: 
  end::UNION{FLOAT64,INT64}


## Avoid non-constant global variables.

In [12]:
# Alright
const global C = 100000
# VERY Bad
global D = 100000

f(x) = x + C
g(x) = x + D

g (generic function with 1 method)

In [6]:
@assert f(10)==g(10)

In [9]:
@timeOften f(10)
@timeOften g(10)

  0.000000 seconds
  0.002352 seconds (100.00 k allocations: 1.526 MB)


Q: Why is a constant so much more efficient?  
A: The compiler cannot infer the type of a dynamic global. A constants type is fixed at declaration.

In [16]:
@code_llvm f(10)


define i64 @julia_f_21855(i64) {
top:
  %1 = add i64 %0, 100000
  ret i64 %1
}


In [11]:
@code_llvm g(10)


define %jl_value_t* @julia_g_21781(i64) {
top:
  %1 = alloca [4 x %jl_value_t*], align 8
  %.sub = getelementptr inbounds [4 x %jl_value_t*]* %1, i64 0, i64 0
  %2 = getelementptr [4 x %jl_value_t*]* %1, i64 0, i64 2
  store %jl_value_t* inttoptr (i64 4 to %jl_value_t*), %jl_value_t** %.sub, align 8
  %3 = load %jl_value_t*** @jl_pgcstack, align 8
  %4 = getelementptr [4 x %jl_value_t*]* %1, i64 0, i64 1
  %.c = bitcast %jl_value_t** %3 to %jl_value_t*
  store %jl_value_t* %.c, %jl_value_t** %4, align 8
  store %jl_value_t** %.sub, %jl_value_t*** @jl_pgcstack, align 8
  store %jl_value_t* null, %jl_value_t** %2, align 8
  %5 = getelementptr [4 x %jl_value_t*]* %1, i64 0, i64 3
  store %jl_value_t* null, %jl_value_t** %5, align 8
  %6 = call %jl_value_t* @jl_box_int64(i64 signext %0)
  store %jl_value_t* %6, %jl_value_t** %2, align 8
  %7 = load %jl_value_t** inttoptr (i64 4437588488 to %jl_value_t**), align 8
  store %jl_value_t* %7, %jl_value_t** %5, align 8
  %8 = call %jl_value_t* 